In [2]:
## Evolving from the basics

# -*- coding: utf-8 -*-
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table_experiments as dt
import pandas as pd

from plotly import graph_objs as go
from plotly.graph_objs import *
from dash.dependencies import Input, Output, State

app = dash.Dash()
#server = app.server
#app.title = 'NYC Wi-Fi Hotspots'

# API keys and datasets
mapbox_access_token = 'pk.eyJ1Ijoid3FlcSIsImEiOiJjankwZmJneGgwMms2M2NtMnZuMTFmMmVpIn0.zQ3TV6-pCSbK3xIxm5-ZLg'
map_data = pd.read_csv("global_power_plant_database.csv")

# Selecting only required columns
map_data = map_data[["country", "country_long", "primary_fuel", "other_fuel1","other_fuel2","name", "latitude", "longitude"]].drop_duplicates()

stanley_style='mapbox://styles/wqeq/cjyf3otbb2kof1cph1frfbxc3'


#  Layouts
layout_table = dict(
                    autosize=True,
                    height=500,
                    font=dict(color="#191A1A"),
                    titlefont=dict(color="#191A1A", size='14'),
                    margin=dict(
                                l=35,
                                r=35,
                                b=35,
                                t=45
                                ),
                    hovermode="closest",
                    plot_bgcolor='#fffcfc',
                    paper_bgcolor='#fffcfc',
                    legend=dict(font=dict(size=10), orientation='h'),
                    )
layout_table['font-size'] = '12'
layout_table['margin-top'] = '20'

layout_map = dict(
                  autosize=True,
                  height=500,
                  font=dict(color="#191A1A"),
                  titlefont=dict(color="#191A1A", size='14'),
                  margin=dict(
                              l=35,
                              r=35,
                              b=35,
                              t=45
                              ),
                  hovermode="closest",
                  plot_bgcolor='#fffcfc',
                  paper_bgcolor='#fffcfc',
                  legend=dict(font=dict(size=10), orientation='h'),
                  title='Environmental crisis',
                  mapbox=dict(
                              accesstoken=mapbox_access_token,
                              style=stanley_style,
                              center=dict(
                                          lon=-73.91251,
                                          lat=40.7342
                                          ),
                              zoom=10,
                              )
                  )

# functions
def gen_map(map_data):
    # groupby returns a dictionary mapping the values of the first field
    # 'classification' onto a list of record dictionaries with that
    # classification value.
    return {
        "data": [{
                 "type": "scattermapbox",
                 "lat": list(map_data['latitude']),
                 "lon": list(map_data['longitude']),
                 "hoverinfo": "text",
                 "hovertext": [["primary_fuel: {} \n name: {} ".format(i,j)]
                               for i,j in zip(map_data['primary_fuel'], map_data['name'])],
                 "mode": "markers",
                 "name": list(map_data['primary_fuel']),
                 "marker": {
                 "size": 6,
                 "opacity": 0.7
                 }
                 }],
        "layout": layout_map
}

app.layout = html.Div(
                      html.Div([
                                html.Div(
                                         [
                                          html.H1(children='Maps and Tables',
                                                  className='nine columns'),
                                          html.Img(
                                                   src="https://static.wixstatic.com/media/f1b802_fab4e1df01a7498a925bf13042e369e1~mv2.jpg/v1/fill/w_714,h_179,al_c,lg_1,q_80/f1b802_fab4e1df01a7498a925bf13042e369e1~mv2.webp",
                                                   className='three columns',
                                                   style={
                                                   'height': '16%',
                                                   'width': '16%',
                                                   'float': 'right',
                                                   'position': 'relative',
                                                   'padding-top': 12,
                                                   'padding-right': 0
                                                   },
                                                   ),
                                          
                                          ], className="row"
                                         ),
                                
                                # Selectors
                                html.Div(
                                         [
                    
                                          html.Div(
                                                   [
                                                    html.P('Fuel Type:'),
                                                    dcc.Dropdown(
                                                                 id='type',
                                                                 options= [{'label': str(item),
                                                                           'value': str(item)}
                                                                           for item in set(map_data['primary_fuel'])],
                                                                 multi=True,
                                                                 value=list(set(map_data['primary_fuel']))[0]
                                                                 )
                                                    ],
                                                   className='six columns',
                                                   style={'margin-top': '10'}
                                                   )
                                          ],
                                         className='row'
                                         ),
                                
                                # Map + table + Histogram
                                html.Div(
                                         [
                                          html.Div(
                                                   [
                                                    dcc.Graph(id='map-graph')
                                                    ], className = "six columns"
                                                   ),
                                          html.Div(
                                                   [
                                                    dt.DataTable(
                                                                 rows=map_data.to_dict('records'),
                                                                 columns=map_data.columns,
                                                                 row_selectable=True,
                                                                 filterable=True,
                                                                 sortable=True,
                                                                 #selected_row_indices=[],
                                                                 id='datatable'),
                                                    ],
                                                   style = layout_table,
                                                   className="six columns"
                                                   ),
                                          
                                          html.Div(
                                                   [
                                                    html.P('Developed by Zhehui Yuan & Zoe Zhou', style = {'display': 'inline'}),
                                                
                                                    ], className = "twelve columns",
                                                   style = {'fontSize': 18, 'padding-top': 20}
                                                   )
                                          ], className="row"
                                         )
                                ], className='ten columns offset-by-one'))



@app.callback(
              Output('datatable', 'rows'),
              [Input('type', 'value'),
               ])
def update_selected_row_indices(type):
    map_aux = map_data.copy()
    
    # Type filter
    map_aux = map_aux[map_aux['primary_fuel'].isin(type)]
    rows = map_aux.to_dict('records')
    return rows

@app.callback(
              Output('map-graph', 'figure'),
              [Input('datatable', 'rows')
               ])#Input('datatable', 'selected_row_indices')
def map_selection(rows):#, selected_row_indices
    aux = pd.DataFrame(rows)
    #temp_df = aux.ix[selected_row_indices, :]
    #if len(selected_row_indices) == 0:
        #return gen_map(aux)
    return gen_map(aux)#temp_df

if __name__ == '__main__':
    app.run_server(1111)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:1111/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Aug/2019 12:35:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2019 12:35:24] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2019 12:35:24] "GET /_favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2019 12:35:24] "GET /_dash-layout HTTP/1.1" 200 -
[2019-08-28 12:35:24,943] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\yzhhu\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\yzhhu\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\yzhhu\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\yzhhu\AppData\Local\Continuum\anacond